In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
import leafmap

In [2]:
load_dotenv("local.env")

True

In [3]:
MAP_KEY = os.getenv("NASA_FIRMS_MAP_KEY")
if MAP_KEY is None:
    raise ValueError("NASA FIRMS MAP_KEY not found in environment variables.")

In [4]:
def get_transaction_count():
    txn_url = f"https://firms.modaps.eosdis.nasa.gov/mapserver/mapkey_status/?MAP_KEY={MAP_KEY}"
    count = 0
    try:
        df = pd.read_json(txn_url,  typ='series')
        count = df['current_transactions']
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    return count

In [5]:
def get_wildfires():
    tcount = get_transaction_count()
    print('Our current transaction count is %i' % tcount)

    # Sensor type
    SENSOR = 'VIIRS_SNPP_NRT'
    # SENSOR = 'VIIRS_NOAA20_NRT'
    # SENSOR = 'MODIS_NRT'

    # Date of interest
    # DATE = '2025-04-09'  # Replace with your desired date
    DAYS = '1'

    # Bounding box for Texas
    # BBOX = '-106.645646,25.837377,-93.508292,36.500704'

    # Bounding box for the continental US
    BBOX =  '-125.859375,25.618963,-63.193359,49.378416'

    # Construct the API URL
    url = f'https://firms.modaps.eosdis.nasa.gov/api/area/csv/{MAP_KEY}/{SENSOR}/{BBOX}/{DAYS}'

    start_count = get_transaction_count()
    df_area = pd.read_csv(url)
    end_count = get_transaction_count()

    print(df_area.head())

    print('We used %i transactions.' % (end_count-start_count))

    return df_area

In [6]:
wildfires_df = get_wildfires()

Our current transaction count is 4
   latitude  longitude  bright_ti4  scan  track    acq_date  acq_time  \
0  48.18580  -64.92191      299.55  0.39   0.36  2025-05-15       618   
1  41.56999  -84.04675      295.85  0.75   0.77  2025-05-15       620   
2  42.94596  -82.41591      299.30  0.63   0.72  2025-05-15       620   
3  43.25919  -79.80930      299.35  0.47   0.64  2025-05-15       620   
4  43.26336  -79.81239      295.10  0.47   0.64  2025-05-15       620   

  satellite instrument confidence version  bright_ti5   frp daynight  
0         N      VIIRS          n  2.0NRT      284.69  0.77        N  
1         N      VIIRS          n  2.0NRT      279.97  1.89        N  
2         N      VIIRS          n  2.0NRT      281.08  1.30        N  
3         N      VIIRS          n  2.0NRT      280.92  1.07        N  
4         N      VIIRS          n  2.0NRT      280.94  2.32        N  
We used 4 transactions.


In [7]:
display(wildfires_df)

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight
0,48.18580,-64.92191,299.55,0.39,0.36,2025-05-15,618,N,VIIRS,n,2.0NRT,284.69,0.77,N
1,41.56999,-84.04675,295.85,0.75,0.77,2025-05-15,620,N,VIIRS,n,2.0NRT,279.97,1.89,N
2,42.94596,-82.41591,299.30,0.63,0.72,2025-05-15,620,N,VIIRS,n,2.0NRT,281.08,1.30,N
3,43.25919,-79.80930,299.35,0.47,0.64,2025-05-15,620,N,VIIRS,n,2.0NRT,280.92,1.07,N
4,43.26336,-79.81239,295.10,0.47,0.64,2025-05-15,620,N,VIIRS,n,2.0NRT,280.94,2.32,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1136,46.72927,-119.86828,329.01,0.70,0.70,2025-05-15,1928,N,VIIRS,n,2.0URT,278.07,3.83,D
1137,46.74647,-119.90322,334.99,0.70,0.70,2025-05-15,1928,N,VIIRS,n,2.0URT,277.95,8.90,D
1138,46.74780,-119.90367,342.70,0.70,0.70,2025-05-15,1928,N,VIIRS,n,2.0URT,278.68,8.50,D
1139,46.74968,-119.89430,351.53,0.70,0.70,2025-05-15,1928,N,VIIRS,n,2.0URT,281.05,13.16,D


In [8]:
print(f"Rows: {len(wildfires_df)}")
print(f"Columns: {wildfires_df.columns}")

Rows: 1141
Columns: Index(['latitude', 'longitude', 'bright_ti4', 'scan', 'track', 'acq_date',
       'acq_time', 'satellite', 'instrument', 'confidence', 'version',
       'bright_ti5', 'frp', 'daynight'],
      dtype='object')


In [9]:
wildfires_df['confidence'].unique()

array(['n', 'h', 'l'], dtype=object)

In [10]:
tx_long = -100.0
tx_lat = 31.0
tx_zoom = 6

In [11]:
m = leafmap.Map(center=[tx_lat, tx_long], zoom=tx_zoom,
                    epsg="4326")

In [12]:
import geopandas as gpd
from shapely.geometry import Point
# Convert the DataFrame to a GeoDataFrame
wildfires_df['geometry'] = wildfires_df.apply(lambda x: Point(x['longitude'], x['latitude']), axis=1)
gdf = gpd.GeoDataFrame(wildfires_df, geometry='geometry')
# drop unnecessary columns
gdf = gdf.drop(columns=['longitude', 'latitude'])
# get rows where confidence is greater than 50
gdf = gdf[gdf['confidence'] == 'h']
gdf = gdf.set_crs(epsg=4326)
print(f"Rows: {len(gdf)}")
print(f"Columns: {gdf.columns}")

Rows: 55
Columns: Index(['bright_ti4', 'scan', 'track', 'acq_date', 'acq_time', 'satellite',
       'instrument', 'confidence', 'version', 'bright_ti5', 'frp', 'daynight',
       'geometry'],
      dtype='object')


In [ ]:
# Add the GeoDataFrame to the map
m.add_gdf(gdf, layer_name="Fires")

In [14]:
m

Map(center=[31.0, -100.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…